This kernel will analyze the loans that Parents ask for their Children. We'll see when, why, and where these loans are asked, and what makes them so special.
![](http://)
![](http://)"my
![](http://)mother
![](http://)was
![](http://)my first country
![](http://)the first place i ever lived",

Nayyirah Waheed

# Table of Contents
* [Loans](#first)
* [Education Loans](#second)
* [Nigeria Education Loans](#third)
* [Gender Analysis of Education Loans](#fourth)
* [Gender Analysis of Education Loans, by Country](#fifth)
* [Gender Analysis of All Loans](#sixth)
* [Funding of Loans](#seventh)


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go


import re
import warnings
warnings.filterwarnings('ignore')

child = pd.read_csv('../input/kiva_loans.csv')
child.disbursed_time = pd.to_datetime(child.disbursed_time)
child.posted_time = pd.to_datetime(child.posted_time)
child.funded_time = pd.to_datetime(child.funded_time)

In [ ]:
child.head(3)

## LOANS <a class="anchor" id="first"></a>

In [ ]:
"""Thank you @SRK"""
loan_sec = child['sector'].value_counts()
trace = go.Bar(
    y=loan_sec.index[::-1],
    x=loan_sec.values[::-1],
    orientation = 'h',
    marker=dict(
        color=loan_sec.values[::-1],
        colorscale = 'Picnic',
        reversescale = True
              ),
            )

layout = dict(
    title='Distribution of loans by sector',
    xaxis=dict(
        title='Number of loans'),
            )

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Loans_by_sector')
print("It would be interesting to dig deeper and see what Personal Use refers to")

In [ ]:
loan_activity = child['activity'].value_counts().head(40)
trace = go.Bar(
    x=loan_activity.index[::-1],
    y=loan_activity.values[::-1],
    orientation = 'v',
    marker=dict(
        color=loan_activity.values[::-1],
        colorscale = 'Picnic',
        reversescale = True
              ),
            )

layout = dict(
    title='Distribution of loans by activity (top 40)',
    yaxis=dict(
        title='Number of loans'),
    xaxis=dict(tickfont=dict(size=8))
            )

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Loans_by_activity')

## Education Loans <a class="anchor" id="second"></a> 
* all education loans that include our Children, Sons, and Daughters

In [ ]:
edu_all = child[child['sector']=='Education']

# find rows with the token child, son, daughter
c_child = edu_all['use'].apply(lambda x: len(re.findall(r"(child)", str(x)))) 
c_son = edu_all['use'].apply(lambda x: len(re.findall(r"([\s.,:;]son[\s.,:;s])|([\s.,:;]son\s*[.'’]+)", str(x)))) 
c_daughter = edu_all['use'].apply(lambda x: len(re.findall(r"([\s.,:;]daughter[\s.,:;s])|([\s.,:;]daughter\s*[.'’]+)", str(x))))

c_child = c_child[c_child.values>0].index
c_son = c_son[c_son.values>0].index
c_daughter = c_daughter[c_daughter.values>0].index

# combine indices of all loans with son, daughter, or child in their description
children = c_child.union(c_son)
children = children.union(c_daughter)

edu_child = child.iloc[children,:]

In [ ]:
edu_for_everyone = child[child['sector']=='Education']['country'].value_counts()
edu_for_everyone = pd.DataFrame(edu_for_everyone).reset_index()
edu_for_everyone.columns = ['country', 'num_all_loans']

edu_for_children = edu_child['country'].value_counts()
edu_for_children = pd.DataFrame(edu_for_children).reset_index()
edu_for_children.columns = ['country', 'num_child_loans']

edu = edu_for_children.merge(edu_for_everyone, on='country', how='right')
edu.fillna(0, inplace=True)
edu['percent'] = edu['num_child_loans']/edu['num_all_loans']
edu['other_loans'] = edu['num_all_loans']-edu['num_child_loans']

edu = edu.sort_values(by='num_child_loans', ascending=False)

In [ ]:
edu = edu[:15]
trace1 = go.Bar(
    x=edu['country'],
    y=edu['num_child_loans'],
    name='Child education_loans'
)
trace2 = go.Bar(
    x=edu['country'],
    y=edu['other_loans'],
    name='Non-child education_loans'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title="Education loans for child and non-child related reasons",
    yaxis=dict(title="Number of loans")
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

Lebanon, Vietnam, and Nigeria request more loans for their childrens' education. This might be because of several reasons:
* We know that many loans for children are asked for primary education. Perhaps primary education in these countries is expensive
* Perhaps familes in these countries have more children, which would make it harder to pay for their education
* Maybe they realized that they are more likely to get a loan for children as opposed to adults
* Or maybe it's based on non-education related reasons

## Nigeria Education Loans <a class="anchor" id="third"></a>
* Let's take a deeper look at Nigeria who has the 2nd highest percentage of education loans for children

In [ ]:
nigeria = edu_child.copy()
nigeria = nigeria[nigeria.country=='Nigeria']
print(nigeria.activity.value_counts())

In [ ]:
print('All education loans for children are managed through 1 field partner ID 288: "Babban Gona Farmers Organization"')
print(nigeria.partner_id.value_counts())

In [ ]:
print("The borrowers are overwhelmingly men")
print(nigeria.borrower_genders.value_counts())

Why are they borrowing money for their childrens' education?

In [ ]:
for x in nigeria.use.head(10):
    print (x)

* It turns out that many of Nigerian education loans are highly related to agricultural reasons

**Timescale of Loans (Nigeria)**

In [ ]:
nigeria = edu_child.copy()
nigeria = nigeria[nigeria.country=='Nigeria']
nigeria['num']= 1 #add 1 to each row so we can count number of loans 

disbursed = nigeria.set_index(nigeria['disbursed_time'])
disbursed = disbursed.loc['2016-09':'2017-02'].resample('5D').sum()

posted = nigeria.set_index(nigeria['posted_time'])
posted = posted.loc['2016-09':'2017-02'].resample('5D').sum()

funded = nigeria.set_index(nigeria['funded_time'])
funded = funded.loc['2016-09':'2017-02'].resample('5D').sum()

plt.figure(figsize=(15,5))
plt.plot(disbursed['num'], color='green', label='Disbursed to borrower', marker='o')
plt.plot(posted['num'], color='red', label='Posted on kiva.org', marker='o')
plt.plot(funded['num'], color='blue', label='Funded on kiva.org', marker='o')
plt.legend(loc='upper left')
plt.title("Number of Education loans for Children, in 5-day intervals (Nigeria)")
plt.ylabel("Number of loans")
plt.show()

Apparently, all 2016 education loans for children were disbursed to borrowers in December, despite not being fully funded yet

## Gender Analysis of Education Loans <a class="anchor" id="fourth"></a>

In [ ]:
ed = child[child['sector']=='Education']
son = ed['use'].apply(lambda x: len(re.findall(r"([\s.,:;]son[\s.,:;s])|([\s.,:;]son\s*[.'’]+)", str(x)))) 
daughter = ed['use'].apply(lambda x: len(re.findall(r"([\s.,:;]daughter[\s.,:;s])|([\s.,:;]daughter\s*[.'’]+)", str(x))))

son = son[son.values>0]
daughter = daughter[daughter.values>0]

ed_son = child.iloc[son.index,:]
ed_daughter = child.iloc[daughter.index,:]

In [ ]:
girl = round(ed_daughter.loan_amount.mean(),2)
boy = round(ed_son.loan_amount.mean(),2)

labels = ['Daughters','Sons']
values = [girl,boy]
colors = ['#38e1a3', '#ffeb38']

trace = go.Pie(labels=labels, values=values,
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=1)))
layout = go.Layout(
        title="Average Education Loans (USD) for Sons and Daughters")

fig = go.Figure(data=[trace], layout=layout)

py.iplot(fig, filename='basic_pie_chart')

Hmmmmm... Parents are asking an average of $63 more for sons than for daughters. It would probably be appropriate to give more money to girls, given that they face more challenges than boys, especially when trying to complete their education.
Some of the Challenges girls face in many countries are:
* missing school because they are asked to help with errands around the house
* missing school because they don't have feminine hygiene products
* having little time to complete their homework because they are asked to watch over siblings

## Gender Analysis of Education Loans, by Country <a class="anchor" id="fifth"></a>

In [ ]:
edu_all = child[child['sector']=='Education']

c_son = edu_all['use'].apply(lambda x: len(re.findall(r"([\s.,:;]son[\s.,:;s])|([\s.,:;]son\s*[.'’]+)", str(x)))) 
c_daughter = edu_all['use'].apply(lambda x: len(re.findall(r"([\s.,:;]daughter[\s.,:;s])|([\s.,:;]daughter\s*[.'’]+)", str(x))))

c_son = c_son[c_son.values>0].index
c_daughter = c_daughter[c_daughter.values>0].index

#unite indices of all loans with son, daughter, or child in their description
son_dautr = c_daughter.union(c_son)

#slice original dataset to extract only son_daughter rows
edu_son_dautr = child.iloc[son_dautr,:]


dollars = edu_son_dautr.groupby('country')['loan_amount'].sum()
all_children = pd.DataFrame(dollars).reset_index()

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = all_children['country'],
        locationmode = 'country names',
        z = all_children['loan_amount'],
        text = all_children['country'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.20,"rgb(40, 60, 190)"],[0.40,"rgb(70, 100, 245)"],\
            [0.55,"rgb(90, 120, 245)"],[0.75,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(280,10,30)',  # color of country borders
                width = 0.5                # width of country borders
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Loans (USD)'),
      ) ]

layout = dict(
    title = 'Total Education Loans for sons and daughters, by Country',
    geo = dict(
        showframe = True,                 # frame of full map
        showcoastlines = True,            # coastline of full map
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='education-loans-for-children-world-map')
print("Southern hemisphere is predominantely the main recipient of education loans")

## Gender Analysis of All Loans <a class="anchor" id="sixth"></a>

In [ ]:
b = child['use'].apply(lambda x: len(re.findall(r"([\s.,:;]son[\s.,:;s])|([\s.,:;]son\s*[.'’]+)", str(x)))) 
g = child['use'].apply(lambda x: len(re.findall(r"([\s.,:;]daughter[\s.,:;s])|([\s.,:;]daughter\s*[.'’]+)", str(x)))) 

b = b[b.values>0]
g = g[g.values>0]

b = child.iloc[b.index,:]
g = child.iloc[g.index,:]

In [ ]:
plt.scatter(x=range(g.shape[0]), y=np.sort(g.loan_amount.values))
plt.title('Loan Amount for Daughters')
plt.xlabel('Index')
plt.ylabel('Loan Amount (\$$)')
plt.show()

let's Truncate to eliminate outliers and display historgram for sons and daughters

In [ ]:
# For Girls

# ulimit (4925.0) is the value below which 99% of observations in this group of loan observations fall
# i.e. 99% of observations in this group of loans fall under $4925.0
ulimit = np.percentile(g.loan_amount.values, 99) 
llimit = np.percentile(g.loan_amount.values, 1) # 1% of observations in this group fall below llimit ($125.0)

g['loan_trunc'] = g['loan_amount'].copy()
g['loan_trunc'].loc[g['loan_amount']>ulimit] = ulimit # if loan > $4925, assign 4925 to loan_amount 
g['loan_trunc'].loc[g['loan_amount']<llimit] = llimit # if loan < $125, assign 125 to loan_amount

# For Boys
ulimit = np.percentile(b.loan_amount.values, 99) 
llimit = np.percentile(b.loan_amount.values, 1) # 1% of observations in this group fall below llimit ($125.0)

b['loan_trunc'] = b['loan_amount'].copy()
b['loan_trunc'].loc[b['loan_amount']>ulimit] = ulimit
b['loan_trunc'].loc[b['loan_amount']<llimit] = llimit

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(b.loan_trunc.values, bins=40, kde=False, color='black')
sns.distplot(g.loan_trunc.values, bins=40, kde=False, color='yellow')
plt.title("Distribution of Loan (USD)")
plt.legend(('sons','daughters'))
plt.ylabel("Number of loans")
plt.xlabel("\$$")
plt.show()

We already established above that sons get an average of \$63 more in education loans than daughters. This plot demonstrates that in all sectors combined, sons get more loans than daughters, and the loans peak at rounded amounts such as \$1000, \$2000, \$1500

**Number of Loans by Sector**

In [ ]:
boys = b.sector.value_counts()
girls = g.sector.value_counts()

trace1 = go.Bar(
    x=girls.index,
    y=girls.values,
    name='Loans for daughters'
)
trace2 = go.Bar(
    x=boys.index,
    y=boys.values,
    name='Loans for sons'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title="Loans by sector (son or daughter)",
    yaxis=dict(title="Number of loans")
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar2')

Here again we see how more loans are asked for sons than for daughters. This could be due to families having more sons or investing more in boys, or a few other reasons. Interesting how only in the Housing sector more loans are asked for daughters than sons.

**Value of Loans by Sector and Gender**

In [ ]:
plt.figure(figsize=[15,5])
sns.stripplot(x='sector', y='loan_amount', data=b, jitter=True)
plt.title('Loans in USD for --Sons--')
plt.xticks(rotation=25)
plt.ylabel('Loan(\$$)')
plt.show()

plt.figure(figsize=[15,5])
sns.stripplot(x='sector', y='loan_amount', data=g, jitter=True, order=b.sector.unique())
plt.title('Loans in USD for --Daughters--')
plt.xticks(rotation=25)
plt.ylabel('Loan(\$$)')
plt.show()

It is remarkable that almost no Transportation or Manufacturing loans are asked for daughters.

In [ ]:
for x in g[g.sector=='Transportation']['use'].head(20):
    print (x)

By digging into Transportation loans, we see that daughters are not always the main recipient of the loans.


## Funding of Loans <a class="anchor" id="seventh"></a>


In [ ]:
"""Combine loans for all sons and daughters"""
sons = child['use'].apply(lambda x: len(re.findall(r"([\s.,:;]son[\s.,:;s])|([\s.,:;]son\s*[.'’]+)", str(x)))) 
dtrs = child['use'].apply(lambda x: len(re.findall(r"([\s.,:;]daughter[\s.,:;s])|([\s.,:;]daughter\s*[.'’]+)", str(x)))) 

sons = sons[sons.values>0].index
dtrs= dtrs[dtrs.values>0].index

# unite indices of all loans with son and daughter in their description
son_dtrs = sons.union(dtrs)

#slice original dataset to extract only son_dtrs rows
df = child.iloc[son_dtrs,:]

In [ ]:
df['left_to_fund'] = df.loan_amount - df.funded_amount
child['left_to_fund'] = child.loan_amount - child.funded_amount

dm = df.left_to_fund.mean()
cm = child.left_to_fund.mean()
data = [go.Bar(
            x=['Sons and Daughters', 'All Loans'],
            y=[dm,cm]
    )]

layout = dict(
    title='Average $ Missing for Unfunded Loans',
    yaxis=dict(
        title="(USD)"),
            )

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar2')
print("On average, loans for sons and daughters are $39 short when not fully funded, compared to $59 for all loans")

**Let's look at percentage of unfunded loans by country**

In [ ]:
all_loans = child['country'].value_counts()
all_loans = pd.DataFrame(all_loans).reset_index()
all_loans.columns = ['country', 'num_all_loans']

not_funded = child[child.left_to_fund>0]['country'].value_counts()
not_funded = pd.DataFrame(not_funded).reset_index()
not_funded.columns = ['country', 'unfunded_loans']

percent = all_loans.merge(not_funded, on='country', how='left')
percent['prcnt_unfunded'] = percent.unfunded_loans / percent.num_all_loans
percent['prcnt_unfunded'] = percent['prcnt_unfunded'].apply(lambda x: 0.0 if pd.isnull(x) else x)
percent['prcnt_unfunded'] = round((percent['prcnt_unfunded']*100),2)

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = percent['country'],
        locationmode = 'country names',
        z = percent['prcnt_unfunded'],
        text = percent.country,
        colorscale = [[0,"rgb(5, 10, 172)"],[0.20,"rgb(40, 60, 190)"],[0.40,"rgb(70, 100, 245)"],\
            [0.55,"rgb(90, 120, 245)"],[0.75,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(280,10,30)',  # color of country borders
                width = 0.5                # width of country borders
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = '% of Loans not funded'),
      ) ]

layout = dict(
    title = 'Percentage of Loans Not fully funded',
    geo = dict(
        showframe = True,                 # frame of full map
        showcoastlines = True,            # coastline of full map
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='unfunded loans')
print("36% of loans in the United States end up not fully funded. Perhaps that is due to rarely using a field partner")

**Unfunded Loans by Sector**

In [ ]:
not_fully_funded = child[child.left_to_fund>0]
plt.figure(figsize=[15,7])
sns.violinplot(x='sector', y='left_to_fund', data=not_fully_funded)
plt.ylim((0,4000))
plt.ylabel("USD")
plt.title("Loans not Fully Funded (USD), by sector")
plt.xticks(rotation=30)
plt.show()

Although there are very few loans that don't get fully funded in the **Arts, Wholesale, and Manufacturing**, the median amount of money missing to fully fund their loans is approximately $1000: Double the median amount in other sectors. In **Entertainment,** the median amount of money missing to fully fund loans is a staggering \$3000

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(not_fully_funded.left_to_fund, not_fully_funded.loan_amount)
plt.xlim((0,10000))
plt.xlabel("Missing $ to Fully Fund Loan")
plt.ylabel("Loan Size (USD)")
plt.show()

We see a slight correlation between size of loan and the amount of dollars that the loan fell short by